In [3]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

### Load dataset and split them into train and test set

In [4]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/'
#####################   Modify above path   #########################
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)
X_dis=X
y_dis=y

### Use GridsearchCV to determine Parameters

In [5]:
# from sklearn.model_selection import GridSearchCV
# xgboost_parm_gs = {
#     'max_depth':[2,4,8] ,
#    'n_estimator': [50,100],
#    'learning_rate': [0.1,0.05,0.01]
# }
# xgboost_model_gs = xgboost.XGBClassifier()
# xgboost_gsearch = GridSearchCV(estimator = xgboost_model_gs, 
#                        param_grid = xgboost_parm_gs, 
#                        scoring ='accuracy',
#                        cv = 3,
#                        n_jobs = 8,
#                        verbose=3,
#                        return_train_score=True
#                       )
# xgboost_gsearch.fit(train_x_dis,train_y_dis)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed: 51.1min
[Parallel(n_jobs=8)]: Done  54 out of  54 | elapsed: 192.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                   

In [6]:
# from sklearn.metrics import accuracy_score  
# print("Training dataset Accuracy")
# train_preds = xgboost_gsearch.predict(train_x_dis)
# train_accuracy = accuracy_score(train_y_dis, train_preds) 
# print(train_accuracy)

Training dataset Accuracy
1.0


In [7]:
# print("Testing dataset Accuracy")
# test_preds = xgboost_gsearch.predict(test_x_dis)
# test_accuracy = accuracy_score(test_y_dis, test_preds) 
# print(test_accuracy)

Testing dataset Accuracy
0.426


In [8]:
# xgboost_gsearch.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimator': 50}

### Use K-Fold to estimate the accuracy(47.12%)

In [3]:
xgboost_model_final =xgboost.XGBClassifier(max_depth=4,n_estimators=50,learning_rate=0.1,
                       min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005)

In [10]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(xgboost_model_final, X_dis,y_dis, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 47.12% (1.70%)


### Fit the final model and save it

In [11]:
t0 = time.time()
xgboost_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

done in 317.931s


In [12]:
# joblib.dump(xgboost_model_final,'xgboost_model_final.m')#please change the path

['xgboost_model_final.m']

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/xgboost_model_final.m')